In [40]:
import requests
import json  
import pandas as pd
import numpy as np

In [77]:
response= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations = response.json()

In [78]:
stations[0]

{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
 'id': 'BikePoints_1',
 'url': '/Place/BikePoints_1',
 'commonName': 'River Street , Clerkenwell',
 'placeType': 'BikePoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'TerminalName',
   'sourceSystemKey': 'BikePoints',
   'value': '001023',
   'modified': '2023-09-01T21:46:13.193Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Installed',
   'sourceSystemKey': 'BikePoints',
   'value': 'true',
   'modified': '2023-09-01T21:46:13.193Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Locked',
   'sourceSystemKey': 'BikePoints',
   'value': 'false',
   'modified': '2023-09-01T21:46:13.193Z'},
  {'$type': 'Tfl.

In [79]:
data =  pd.DataFrame()
s_names = []
s_lat = []
s_lon = []

NbBikes = []
NbEmptyDocks = []
NbDocks = []
NbStandardBikes = []
NbEBikes = []
s_id = []

In [83]:
len_lat = 0
for station in stations:
    s_names.append(station['commonName'])
    s_lat.append(float(station['lat']))
    s_lon.append(float(station['lon']))
    for add_property in station['additionalProperties']:
        if add_property['key'] == 'NbBikes':
            NbBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEmptyDocks':
            NbEmptyDocks.append(int(add_property['value']))     
        if add_property['key'] == 'NbDocks':
            NbDocks.append(int(add_property['value']))
        if add_property['key'] == 'NbStandardBikes':
            NbStandardBikes.append(int(add_property['value']))
        if add_property['key'] == 'NbEBikes':
            NbEBikes.append(int(add_property['value']))
        if add_property['key'] == 'TerminalName':
            s_id.append(int(add_property['value']))
        

In [91]:
len(s_lat)

796

In [ ]:
data['Station_name'] = s_names
data['s_lat'] = s_lat
data['s_lon'] = s_lon
data['s_num_bikes'] = NbBikes
data['s_num_empty_docks'] = NbEmptyDocks
data['s_num_std_bikes'] = NbStandardBikes
data['s_num_e-bikes'] = NbEBikes
data['s_num_docks'] = data['s_num_bikes'] + data['s_num_empty_docks']
data['s_id'] = s_id

In [ ]:
data

In [ ]:
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

In [ ]:
loc = []

In [ ]:
for station in data.index:
    if data['s_lat'][station] > c_toplimit_lat and data['s_lat'][station]< c_lowlimit_lat and data['s_lon'][station]< c_toplimit_lon and data['s_lon'][station]>c_lowlimit_lon:
        loc.append('Central')
    elif data['s_lon'][station] < n_toplimit_lon and data['s_lon'][station] > n_lowlimit_lon and data['s_lat'][station] > n_lowlimit_lat:
        loc.append('North')
    elif data['s_lon'][station] < w_limit_lon and data['s_lat'][station] > w_limit_lat:
        loc.append('West')
    elif data['s_lon'][station] < sw_limit_lon and data['s_lat'][station] < sw_limit_lat:
        loc.append('South_West')  
    elif data['s_lon'][station] > e_limit_lon and data['s_lat'][station] > e_limit_lat:
        loc.append('East')     
    elif data['s_lon'][station] > se_limit_lon and data['s_lat'][station] < se_limit_lat:
        loc.append('East')    
    else:
        loc.append('Other')

In [ ]:
data['Area_loc'] = loc

In [ ]:
data

In [ ]:
stations_details=data['Station_name'].str.split(', ', expand=True)
stations=stations_details.rename(columns={0:'Street',1:'Neighbourhood'})

In [ ]:
stations_fv=stations[['Street','Neighbourhood']]
stations_fv['Street']=stations_fv['Street'].apply(lambda x: x.strip())
# stations_fv['Neighbourhood']=stations_fv['Neighbourhood'].apply(lambda x: x.strip())

In [ ]:
stations_fv['Neighbourhood'].unique()

In [ ]:
stations_fv['Station_name']=data['Station_name']

In [ ]:
df_stations=data.merge(stations_fv,on='Station_name')

In [ ]:
df_stations

In [ ]:
df_stations.columns

In [ ]:
stations_df_st=df_stations[['s_id','Station_name','Street',
                            'Neighbourhood','Area_loc', 's_lat', 's_lon','s_num_docks']]

In [ ]:
stations_df_st

### Colour Formatting

In [ ]:
color_dict={'Area_loc': list(stations_df_st['Area_loc'].unique())
                       , 'Colours': ['#ffbaba','#ff7b7b', '#ff5252', '#ff0000', '#a70000']
                    ,'index':[0, 1, 2, 3,4]}

color_df=pd.DataFrame(color_dict)
color_df


In [ ]:
stations_df_st.columns

In [ ]:
stations_df_st=stations_df_st.merge(color_df,on='Area_loc',how='left')

In [ ]:
stations_df_st

### Size Bucket

In [ ]:

stations_df_st['Size_bucket']=[(50 if x<=15 else (100 if 15<x<=30 else (150 if 30<x<=45 else 200))) for x in stations_df_st['s_num_docks']]



In [ ]:
path_st='raw_data/stations_df_st.csv'

In [ ]:
stations_df_st.to_csv(path_st)

In [ ]:
stations_df_st

In [ ]:
stations_df_st[stations_df_st.Neighbourhood=='Liverpool Street']

## Calling the API

In [41]:
response_2= requests.get('https://api.tfl.gov.uk/BikePoint/')
stations_2 = response_2.json()

In [48]:
stations_2[0]

{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
 'id': 'BikePoints_1',
 'url': '/Place/BikePoints_1',
 'commonName': 'River Street , Clerkenwell',
 'placeType': 'BikePoint',
 'additionalProperties': [{'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'TerminalName',
   'sourceSystemKey': 'BikePoints',
   'value': '001023',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Installed',
   'sourceSystemKey': 'BikePoints',
   'value': 'true',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.Api.Presentation.Entities.AdditionalProperties, Tfl.Api.Presentation.Entities',
   'category': 'Description',
   'key': 'Locked',
   'sourceSystemKey': 'BikePoints',
   'value': 'false',
   'modified': '2023-09-01T21:08:07.183Z'},
  {'$type': 'Tfl.

In [46]:
origin='Shoreditch Park, Hoxton'
destination='Finsbury Circus, Liverpool Street'

In [89]:
for station in stations_2:
    if station['commonName']==origin:
        for add_property in station['additionalProperties']:
            if add_property['key'] == 'NbBikes':
                print(add_property['value'])
    if station['commonName']==destination:
            for add_property in station['additionalProperties']:
                if add_property['key'] == 'NbEmptyDocks':
                    print(add_property['value'])
        


12
18


In [93]:
for station in stations_2:
    if station['commonName']==origin:
        print(station['lat'])
        print(station['lon'])
    if station['commonName']==destination:
        print(station['lat'])
        print(station['lon'])
        

51.517075
-0.086685
51.534042
-0.086379
